Note: Oct 2 Adding support for dates, the rest of the ipyn still need to be edited to allow for this. 

Days Scraped: Oct2 - 3:06pm

In [10]:
from  bs4 import BeautifulSoup
import requests
import pandas as pd
import re


In [2]:
att_iphones_url = "https://swappa.com/mobile/att/iphones"
models = ['apple-iphone-x-a1865','apple-iphone-se', 'apple-iphone-8-plus-a1864', 'apple-iphone-8-a1863',
          'apple-iphone-7-plus-a1661','apple-iphone-7-a1660','apple-iphone-6s-plus','apple-iphone-6s']
carriers = ['verizon', 'att', 'tmobile','sprint',]

In [3]:
def pull_sold_data(link):
    url = 'https://swappa.com' + link
    page = requests.get(url).text
    soup = BeautifulSoup(page, features="html5lib")
    recently_sold = soup.findAll(class_='table table-bordered table-condensed table-centered')
    date = re.sub('\s+', '', recently_sold[0].findAll('td')[6].text)
    return(date)

In [4]:
def sale_details(url):
    
    page = requests.get(url).text
    soup = BeautifulSoup(page, features="html5lib")
    recently_sold = soup.findAll(class_='listing_row listing_ listing_')
    
    phone_data = []
    for i in range(len(recently_sold)):
        condition = recently_sold[i].find(class_="condition_label").text
        color = recently_sold[i].find(class_="color_label").text
        storage = recently_sold[i].find(class_="storage_label").text
        sold_price = recently_sold[i].find(class_="price").text
        
        link = recently_sold[i].find('a')['href']
        date = pull_sold_data(link)
    
        phone_data.append([condition,color,int(storage.strip('GB')),int(sold_price.strip('$')),date])
        
    return(phone_data)

In [5]:
print(sale_details("https://swappa.com/buy/apple-iphone-x-a1865-att"))

[['Mint', 'Gray', 256, 879, 'Oct02,2018-03:19PM'], ['Mint', 'Gray', 64, 695, 'Oct01,2018-10:48AM'], ['Mint', 'Gray', 256, 765, 'Oct01,2018-01:01AM'], ['Mint', 'Gray', 256, 850, 'Sep29,2018-02:32PM'], ['Mint', 'Gray', 64, 749, 'Sep29,2018-12:03PM']]


In [6]:
col_names = ['Phone Model', 'Carrier', 'Condition', 'Color', 'Size', 'Recently Sold Price','Date']
iphone_df = pd.DataFrame(columns=col_names)
print(iphone_df)

Empty DataFrame
Columns: [Phone Model, Carrier, Condition, Color, Size, Recently Sold Price, Date]
Index: []


In [7]:
for model in models:
    for carrier in carriers:
        url = "https://swappa.com/buy/"+ model + '-' + carrier
        data = sale_details(url)
            
        for sold in data:
            iphone_df.loc[len(iphone_df)] = [model, carrier, sold[0],sold[1],sold[2],sold[3] ,sold[4]]


In [8]:
print(iphone_df.head(10))

            Phone Model  Carrier Condition   Color Size Recently Sold Price  \
0  apple-iphone-x-a1865  verizon      Good    Gray  256                 650   
1  apple-iphone-x-a1865  verizon      Mint    Gray  256                 759   
2  apple-iphone-x-a1865  verizon      Good  Silver  256                 670   
3  apple-iphone-x-a1865  verizon      Mint    Gray  256                 735   
4  apple-iphone-x-a1865  verizon      Fair  Silver   64                 644   
5  apple-iphone-x-a1865      att      Mint    Gray  256                 879   
6  apple-iphone-x-a1865      att      Mint    Gray   64                 695   
7  apple-iphone-x-a1865      att      Mint    Gray  256                 765   
8  apple-iphone-x-a1865      att      Mint    Gray  256                 850   
9  apple-iphone-x-a1865      att      Mint    Gray   64                 749   

                 Date  
0  Oct02,2018-04:18PM  
1  Oct02,2018-03:57PM  
2  Oct02,2018-01:20PM  
3  Oct02,2018-12:12PM  
4  Oct02,2

In [9]:
file_name = "iphone_df_Oct_2_w_dates.csv"
iphone_df.to_csv(file_name, encoding='utf-8', index=False)